データサイエンス特別PG データサイエンス特論 第10回 課題

In [ ]:
# google colab で実行する場合は下記のコメントアウトを外して実行する
# ! wget https://github.com/KHiraGit/sudspg_ds/raw/main/ds07_temp_power_exercise.xlsx
# ! pip install japanize-matplotlib

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np
import pandas as pd
import datetime
import sklearn
import openpyxl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble  import RandomForestClassifier
from sklearn.metrics import f1_score
import seaborn as sns

In [ ]:
weather_df = pd.read_excel('ds07_temp_power_exercise.xlsx', 
                           sheet_name='2020熊谷市気象データ_data', skiprows=[0,1,2],
                           names=['年月日', '平均気温', '', '', '最高気温', '', '', '', '', '最低気温', '', '', '', '', 
                           '降水量', '', '', '', '日照時間', '', '', '', '降雪量', '', '', '', '平均風速', '', '', 
                           '平均蒸気圧', '', '', '平均湿度', '', '', '平均現地気圧', '', '', '', '', '', '天気概況', '', ''])

year = 2020
month = 1
data = ['平均気温', '最高気温', '最低気温', '降水量', '日照時間', '降雪量', '平均風速', '平均蒸気圧', '平均湿度', '平均現地気圧'] 
target = '天気概況'
weather_data = weather_df[(datetime.datetime(year,month,1) <= weather_df['年月日']) 
                        & (weather_df['年月日'] < datetime.datetime(year,month+1,1))][data].values
weather_target = weather_df[(datetime.datetime(year,month,1) <= weather_df['年月日']) 
                            & (weather_df['年月日'] < datetime.datetime(year,month+1,1))][target].values

In [ ]:
_max_depth = 3
clf = DecisionTreeClassifier(criterion='entropy', max_depth=_max_depth)
clf = clf.fit(weather_data, weather_target)
print(f'max_depth={_max_depth}')
print('f1_score:', f1_score(weather_target, clf.predict(weather_data), average="micro"))
plt.figure(figsize=(15, 10))
plot_tree(clf, filled=True, feature_names=data, class_names=weather_target, fontsize=12)
plt.show()

In [ ]:
# ランダムフォレストを作成してF値を計算 (木の数、木の深さはグリッドサーチ)
best_f1_score = 0
best_estimator = 0
best_depth = 0
best_params = []
_heatmap = []
for _depth in range(1,6):
    _heatmap_row = []
    for _estimator in range(1,51):
        rf = RandomForestClassifier(n_estimators=_estimator, max_depth=_depth, random_state=1)
        rf = rf.fit(weather_data, weather_target)
        _f1_score = f1_score(weather_target, rf.predict(weather_data), average="micro")
        _heatmap_row.append(_f1_score)
        if _f1_score > best_f1_score:
            best_f1_score = _f1_score
            best_estimator = _estimator
            best_depth = _depth
            if _f1_score == 1.0:
                best_params.append([_estimator, _depth])
    _heatmap.append(_heatmap_row)

print(f'best n_estimators={best_estimator}')
print(f'best depth={best_depth}')
print(f'best f1_score={best_f1_score}')
print(f'params of f1_score=1.0 -> {best_params}')

In [ ]:
# 説明変数の重要度をグラフに出力
rf = RandomForestClassifier(n_estimators=best_estimator, max_depth=best_depth, random_state=1)
rf = rf.fit(weather_data, weather_target)
importances = rf.feature_importances_
plt.figure(figsize=(12, 5))
plt.barh(range(len(data)), rf.feature_importances_ , align='center')
plt.yticks(np.arange(len(data)), data)
plt.show()